In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AI_hack/data/Protenus Sales Training Data.csv")
df.head()

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,Account_Fiscal_Year_End,Account_State,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue
0,10/1/19,0064100000TL1Fw,Stage 3 - Pricing,New Business,NaN,9/12/18,11/12/19,Diversion Monitoring,Epic,Childrens Hospital,12/31/23,OH,43205.0,2.0,694.0,2196.0,7282.0,85077078.0,3.652792e+09,2.048661e+09
1,10/1/19,0064100000TKWAx,Stage 3 - Pricing,New Business,NaN,9/7/18,12/12/19,Privacy Monitoring,Epic,Health System,NaN,HI,96813.0,4.0,602.0,1829.0,4443.0,-242188.0,3.560499e+09,1.340713e+09
2,10/1/19,0064100000TKw69,Stage 2 - Scoping,New Business,NaN,9/11/18,4/15/20,Diversion Monitoring,Epic,Health System,NaN,MI,49503.0,10.0,1901.0,4000.0,NaN,982985681.0,7.869236e+09,3.290488e+09
3,10/1/19,0064100000TK0q9,Stage 2 - Scoping,New Business,NaN,8/29/18,2/22/20,Diversion Monitoring,Epic,Health System,NaN,OH,43604.0,12.0,1594.0,2481.0,NaN,165601560.0,1.075654e+10,2.468940e+09
4,10/1/19,0064100000QhXXx,Stage 3 - Pricing,New Business,NaN,6/26/18,12/10/19,Privacy Monitoring,Epic,Health System,NaN,WI,53226.0,10.0,1259.0,2666.0,8197.0,20765609.0,1.024796e+10,3.881722e+09


### Random Forest

In [ ]:
#Convert date columns to datetime
df['Snapshot_Date'] = pd.to_datetime(df['Snapshot_Date'])
df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'])
df['Opportunity_Close_Date'] = pd.to_datetime(df['Opportunity_Close_Date'])

In [ ]:
#Q3 of 2023
Q3_2023_start = datetime(2023, 7, 1)
Q3_2023_end = datetime(2023, 9, 30)

In [ ]:
df['Target_Close_Q3_2023'] = df['Opportunity_Close_Date'].apply(lambda x: 1 if Q3_2023_start <= x <= Q3_2023_end else 0)
df['Opportunity_Duration'] = (df['Opportunity_Close_Date'] - df['Opportunity_Created_Date']).dt.days

In [ ]:
missing_values = df.isnull().sum()
missing_values

Snapshot_Date                                  0
Opportunity_ID                                 0
Opportunity_Stage                              0
Opportunity_Type                               0
Opportunity_Annual_Amount                   1238
Opportunity_Created_Date                       0
Opportunity_Close_Date                         0
Opportunity_Product                            0
Account_Electronic_Medical_Record_System       0
Account_Hospital_Type                        646
Account_Fiscal_Year_End                     6651
Account_State                                630
Account_Zip_Code                             646
Account_Number_of_Hospitals                 2119
Account_Number_of_Beds                       666
Account_Number_of_Affiliated_Physicians      706
Account_Number_of_Employee                  3040
Account_Cash_on_Hand                         920
Account_Total_Patient_Revenue                827
Account_Net_Patient_Revenue                  827
Target_Close_Q3_2023

In [ ]:
df.head()

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,...,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue,Target_Close_Q3_2023,Opportunity_Duration
0,2019-10-01,0064100000TL1Fw,Stage 3 - Pricing,New Business,NaN,2018-09-12,2019-11-12,Diversion Monitoring,Epic,Childrens Hospital,...,43205.0,2.0,694.0,2196.0,7282.0,85077078.0,3.652792e+09,2.048661e+09,0,426
1,2019-10-01,0064100000TKWAx,Stage 3 - Pricing,New Business,NaN,2018-09-07,2019-12-12,Privacy Monitoring,Epic,Health System,...,96813.0,4.0,602.0,1829.0,4443.0,-242188.0,3.560499e+09,1.340713e+09,0,461
2,2019-10-01,0064100000TKw69,Stage 2 - Scoping,New Business,NaN,2018-09-11,2020-04-15,Diversion Monitoring,Epic,Health System,...,49503.0,10.0,1901.0,4000.0,NaN,982985681.0,7.869236e+09,3.290488e+09,0,582
3,2019-10-01,0064100000TK0q9,Stage 2 - Scoping,New Business,NaN,2018-08-29,2020-02-22,Diversion Monitoring,Epic,Health System,...,43604.0,12.0,1594.0,2481.0,NaN,165601560.0,1.075654e+10,2.468940e+09,0,542
4,2019-10-01,0064100000QhXXx,Stage 3 - Pricing,New Business,NaN,2018-06-26,2019-12-10,Privacy Monitoring,Epic,Health System,...,53226.0,10.0,1259.0,2666.0,8197.0,20765609.0,1.024796e+10,3.881722e+09,0,532


In [ ]:
df.drop(["Account_Fiscal_Year_End"], axis=1)
df = df.dropna(subset=['Opportunity_Annual_Amount'])
df

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,...,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue,Target_Close_Q3_2023,Opportunity_Duration
136,2019-11-01,0064100000a4MNO,Closed Lost,New Business,239333.4085,2018-10-08,2019-10-01,Diversion Monitoring,Epic,Health System,...,60201.0,6.0,1429.0,3596.0,6071.0,60357592.0,9.389305e+09,2.649645e+09,0,358
147,2019-11-01,0061K00000e2RhD,Closed Won,Existing Business,0.0000,2019-09-09,2019-10-16,Privacy Monitoring,Epic,Critical Access Hospital,...,83110.0,NaN,22.0,64.0,216.0,13320226.0,9.414469e+07,6.601294e+07,0,37
163,2019-11-01,0061K00000e0yVq,Closed Lost,New Business,100127.5051,2019-08-06,2019-10-08,Privacy Monitoring,Meditech,Short Term Acute Care Hospital,...,28546.0,25.0,87.0,305.0,977.0,14367809.0,4.708955e+08,1.553847e+08,0,63
195,2019-11-01,0061K00000d0aGr,Closed Lost,New Business,159879.6365,2019-06-24,2019-10-17,Privacy Monitoring,Epic,Health System,...,98034.0,2.0,330.0,828.0,3557.0,68435213.0,2.383208e+09,8.362493e+08,0,115
208,2019-11-01,0061K00000cj6LD,Closed Lost,New Business,102378.0091,2019-04-12,2019-10-02,Privacy Monitoring,Epic,Short Term Acute Care Hospital,...,68901.0,NaN,97.0,162.0,946.0,1058767.0,5.082591e+08,2.000436e+08,0,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8839,2023-07-01,0061K00000h6jwx,Stage 3 - Pricing,New Business,75578.9900,2021-03-03,2024-01-18,Diversion Monitoring,Cerner,Health System,...,33759.0,18.0,3507.0,4506.0,16328.0,2697970.0,1.840575e+10,4.206860e+09,0,1051
8840,2023-07-01,0061K00000fvP5f,Stage 3 - Pricing,Existing Business,81073.3200,2020-09-16,2023-11-01,Diversion Monitoring,Epic,Health System,...,98405.0,12.0,1900.0,4731.0,11550.0,40220179.0,1.387297e+10,3.600545e+09,0,1141
8841,2023-07-01,0061K00000ftCbz,Stage 3 - Pricing,New Business,33890.0000,2020-07-22,2023-12-23,Diversion Monitoring,Other,Health System,...,85016.0,2.0,352.0,1190.0,4228.0,190886512.0,4.091840e+09,1.255390e+09,0,1249
8842,2023-07-01,0061K00000fNEfL,Stage 1 - Evaluating,Existing Business,30240.0000,2020-05-22,2024-05-23,Privacy Monitoring,Epic,Health System,...,27710.0,4.0,1553.0,3886.0,NaN,NaN,1.303975e+10,3.875812e+09,0,1462


In [ ]:
X = df[["Opportunity_Stage", "Opportunity_Type", "Opportunity_Product", "Opportunity_Duration"]]
y = df['Target_Close_Q3_2023']

In [ ]:
X

,Opportunity_Stage,Opportunity_Type,Opportunity_Product,Opportunity_Duration
136,Closed Lost,New Business,Diversion Monitoring,358
147,Closed Won,Existing Business,Privacy Monitoring,37
163,Closed Lost,New Business,Privacy Monitoring,63
195,Closed Lost,New Business,Privacy Monitoring,115
208,Closed Lost,New Business,Privacy Monitoring,173
...,...,...,...,...
8839,Stage 3 - Pricing,New Business,Diversion Monitoring,1051
8840,Stage 3 - Pricing,Existing Business,Diversion Monitoring,1141
8841,Stage 3 - Pricing,New Business,Diversion Monitoring,1249
8842,Stage 1 - Evaluating,Existing Business,Privacy Monitoring,1462


In [ ]:
one_hot_X = pd.get_dummies(X)
one_hot_X

,Opportunity_Duration,Opportunity_Stage_Closed Lost,Opportunity_Stage_Closed Won,Opportunity_Stage_Stage 0 - Prospecting,Opportunity_Stage_Stage 1 - Evaluating,Opportunity_Stage_Stage 2 - Scoping,Opportunity_Stage_Stage 3 - Pricing,Opportunity_Stage_Stage 4 - Verbal / VOC,Opportunity_Stage_Stage 5 - Contracting,Opportunity_Type_Existing Business,Opportunity_Type_New Business,Opportunity_Product_Compliance Analytics,Opportunity_Product_Diversion Monitoring,Opportunity_Product_Privacy Monitoring
136,358,1,0,0,0,0,0,0,0,0,1,0,1,0
147,37,0,1,0,0,0,0,0,0,1,0,0,0,1
163,63,1,0,0,0,0,0,0,0,0,1,0,0,1
195,115,1,0,0,0,0,0,0,0,0,1,0,0,1
208,173,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8839,1051,0,0,0,0,0,1,0,0,0,1,0,1,0
8840,1141,0,0,0,0,0,1,0,0,1,0,0,1,0
8841,1249,0,0,0,0,0,1,0,0,0,1,0,1,0
8842,1462,0,0,0,1,0,0,0,0,1,0,0,0,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_smote, y_train_smote)

y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

classification_report_rf = classification_report(y_test, y_pred_rf, target_names=['Not Close', 'Close'])
roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)

### Prediction

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AI_hack/data/Protenus Sales Training Data.csv")
df.head()

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,Account_Fiscal_Year_End,Account_State,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue
0,10/1/19,0064100000TL1Fw,Stage 3 - Pricing,New Business,NaN,9/12/18,11/12/19,Diversion Monitoring,Epic,Childrens Hospital,12/31/23,OH,43205.0,2.0,694.0,2196.0,7282.0,85077078.0,3.652792e+09,2.048661e+09
1,10/1/19,0064100000TKWAx,Stage 3 - Pricing,New Business,NaN,9/7/18,12/12/19,Privacy Monitoring,Epic,Health System,NaN,HI,96813.0,4.0,602.0,1829.0,4443.0,-242188.0,3.560499e+09,1.340713e+09
2,10/1/19,0064100000TKw69,Stage 2 - Scoping,New Business,NaN,9/11/18,4/15/20,Diversion Monitoring,Epic,Health System,NaN,MI,49503.0,10.0,1901.0,4000.0,NaN,982985681.0,7.869236e+09,3.290488e+09
3,10/1/19,0064100000TK0q9,Stage 2 - Scoping,New Business,NaN,8/29/18,2/22/20,Diversion Monitoring,Epic,Health System,NaN,OH,43604.0,12.0,1594.0,2481.0,NaN,165601560.0,1.075654e+10,2.468940e+09
4,10/1/19,0064100000QhXXx,Stage 3 - Pricing,New Business,NaN,6/26/18,12/10/19,Privacy Monitoring,Epic,Health System,NaN,WI,53226.0,10.0,1259.0,2666.0,8197.0,20765609.0,1.024796e+10,3.881722e+09


In [ ]:
# Converting 'Snapshot_Date' to datetime for sorting and filtering
df['Snapshot_Date'] = pd.to_datetime(df['Snapshot_Date'])

# Sorting the dataframe by 'Opportunity_ID' and 'Snapshot_Date'
# This will ensure that the latest snapshot of each opportunity comes last
df_sorted = df.sort_values(by=['Opportunity_ID', 'Snapshot_Date'])

# Dropping duplicates, keeping the last occurrence (latest snapshot)
df_filtered = df_sorted.drop_duplicates(subset='Opportunity_ID', keep='last')

# Displaying the first few rows of the filtered dataframe
df_filtered.head()

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,Account_Fiscal_Year_End,Account_State,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue
3059,2021-04-01,0061K00000b3JMy,Closed Lost,New Business,167838.3211,3/5/19,3/30/21,Privacy Monitoring,Other,Short Term Acute Care Hospital,6/30/23,PA,15243.0,NaN,296.0,684.0,NaN,21723248.0,1.471406e+09,3.705945e+08
8063,2023-01-01,0061K00000b3Qsm,Closed Won,New Business,383667.8517,3/7/19,12/2/22,Privacy Monitoring,Epic,Health System,NaN,TN,38120.0,21.0,2708.0,3670.0,12464.0,160091963.0,1.345325e+10,2.802628e+09
386,2019-12-01,0061K00000b3uDh,Closed Lost,New Business,177324.3041,3/13/19,11/11/19,Privacy Monitoring,Epic,Short Term Acute Care Hospital,6/30/23,CO,81003.0,NaN,268.0,596.0,2382.0,7557239.0,2.771359e+09,4.952779e+08
405,2020-01-01,0061K00000bI3DY,Closed Won,New Business,224617.8303,11/20/18,12/12/19,Privacy Monitoring,Epic,Childrens Hospital,8/31/23,IL,60611.0,NaN,364.0,1675.0,NaN,123858245.0,3.091602e+09,1.039410e+09
383,2019-12-01,0061K00000bJS0n,Closed Lost,New Business,166841.1412,12/14/18,11/4/19,Diversion Monitoring,Epic,Health System,NaN,TX,75235.0,2.0,449.0,1208.0,3592.0,181059081.0,4.296671e+09,1.895079e+09


In [ ]:
df_filtered['Snapshot_Date'] = pd.to_datetime(df_filtered['Snapshot_Date'])
df_filtered['Opportunity_Created_Date'] = pd.to_datetime(df_filtered['Opportunity_Created_Date'])
df_filtered['Opportunity_Close_Date'] = pd.to_datetime(df_filtered['Opportunity_Close_Date'])

<ipython-input-476-c76d631f1f36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Snapshot_Date'] = pd.to_datetime(df_filtered['Snapshot_Date'])
<ipython-input-476-c76d631f1f36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Opportunity_Created_Date'] = pd.to_datetime(df_filtered['Opportunity_Created_Date'])
<ipython-input-476-c76d631f1f36>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [ ]:
Q3_2023_start = datetime(2023, 7, 1)
Q3_2023_end = datetime(2023, 9, 30)

In [ ]:
df_filtered['Target_Close_Q3_2023'] = df_filtered['Opportunity_Close_Date'].apply(lambda x: 1 if Q3_2023_start <= x <= Q3_2023_end else 0)
df_filtered['Opportunity_Duration'] = (df_filtered['Opportunity_Close_Date'] - df_filtered['Opportunity_Created_Date']).dt.days

<ipython-input-478-ccf6db88886e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Target_Close_Q3_2023'] = df_filtered['Opportunity_Close_Date'].apply(lambda x: 1 if Q3_2023_start <= x <= Q3_2023_end else 0)
<ipython-input-478-ccf6db88886e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Opportunity_Duration'] = (df_filtered['Opportunity_Close_Date'] - df_filtered['Opportunity_Created_Date']).dt.days


In [ ]:
df_filtered

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,...,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue,Target_Close_Q3_2023,Opportunity_Duration
3059,2021-04-01,0061K00000b3JMy,Closed Lost,New Business,167838.3211,2019-03-05,2021-03-30,Privacy Monitoring,Other,Short Term Acute Care Hospital,...,15243.0,NaN,296.0,684.0,NaN,21723248.0,1.471406e+09,3.705945e+08,0,756
8063,2023-01-01,0061K00000b3Qsm,Closed Won,New Business,383667.8517,2019-03-07,2022-12-02,Privacy Monitoring,Epic,Health System,...,38120.0,21.0,2708.0,3670.0,12464.0,160091963.0,1.345325e+10,2.802628e+09,0,1366
386,2019-12-01,0061K00000b3uDh,Closed Lost,New Business,177324.3041,2019-03-13,2019-11-11,Privacy Monitoring,Epic,Short Term Acute Care Hospital,...,81003.0,NaN,268.0,596.0,2382.0,7557239.0,2.771359e+09,4.952779e+08,0,243
405,2020-01-01,0061K00000bI3DY,Closed Won,New Business,224617.8303,2018-11-20,2019-12-12,Privacy Monitoring,Epic,Childrens Hospital,...,60611.0,NaN,364.0,1675.0,NaN,123858245.0,3.091602e+09,1.039410e+09,0,387
383,2019-12-01,0061K00000bJS0n,Closed Lost,New Business,166841.1412,2018-12-14,2019-11-04,Diversion Monitoring,Epic,Health System,...,75235.0,2.0,449.0,1208.0,3592.0,181059081.0,4.296671e+09,1.895079e+09,0,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2021-02-01,0064100000TKw69,Closed Lost,New Business,412829.9260,2018-09-11,2021-01-26,Diversion Monitoring,Epic,Health System,...,49503.0,10.0,1901.0,4000.0,NaN,982985681.0,7.869236e+09,3.290488e+09,0,868
6680,2022-08-01,0064100000TL1Fw,Closed Lost,New Business,317822.6165,2018-09-12,2022-07-07,Diversion Monitoring,Epic,Childrens Hospital,...,43205.0,2.0,694.0,2196.0,7282.0,85077078.0,3.652792e+09,2.048661e+09,0,1394
136,2019-11-01,0064100000a4MNO,Closed Lost,New Business,239333.4085,2018-10-08,2019-10-01,Diversion Monitoring,Epic,Health System,...,60201.0,6.0,1429.0,3596.0,6071.0,60357592.0,9.389305e+09,2.649645e+09,0,358
743,2020-05-01,0064100000a6Ou1,Closed Lost,New Business,348860.0021,2018-10-30,2020-04-24,Diversion Monitoring,Epic,Health System,...,52242.0,2.0,922.0,1854.0,8485.0,15469189.0,6.972739e+09,2.113914e+09,0,542


In [ ]:
df_filtered.drop(["Account_Fiscal_Year_End"], axis=1)
df_filtered

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,...,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue,Target_Close_Q3_2023,Opportunity_Duration
3059,2021-04-01,0061K00000b3JMy,Closed Lost,New Business,167838.3211,2019-03-05,2021-03-30,Privacy Monitoring,Other,Short Term Acute Care Hospital,...,15243.0,NaN,296.0,684.0,NaN,21723248.0,1.471406e+09,3.705945e+08,0,756
8063,2023-01-01,0061K00000b3Qsm,Closed Won,New Business,383667.8517,2019-03-07,2022-12-02,Privacy Monitoring,Epic,Health System,...,38120.0,21.0,2708.0,3670.0,12464.0,160091963.0,1.345325e+10,2.802628e+09,0,1366
386,2019-12-01,0061K00000b3uDh,Closed Lost,New Business,177324.3041,2019-03-13,2019-11-11,Privacy Monitoring,Epic,Short Term Acute Care Hospital,...,81003.0,NaN,268.0,596.0,2382.0,7557239.0,2.771359e+09,4.952779e+08,0,243
405,2020-01-01,0061K00000bI3DY,Closed Won,New Business,224617.8303,2018-11-20,2019-12-12,Privacy Monitoring,Epic,Childrens Hospital,...,60611.0,NaN,364.0,1675.0,NaN,123858245.0,3.091602e+09,1.039410e+09,0,387
383,2019-12-01,0061K00000bJS0n,Closed Lost,New Business,166841.1412,2018-12-14,2019-11-04,Diversion Monitoring,Epic,Health System,...,75235.0,2.0,449.0,1208.0,3592.0,181059081.0,4.296671e+09,1.895079e+09,0,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2021-02-01,0064100000TKw69,Closed Lost,New Business,412829.9260,2018-09-11,2021-01-26,Diversion Monitoring,Epic,Health System,...,49503.0,10.0,1901.0,4000.0,NaN,982985681.0,7.869236e+09,3.290488e+09,0,868
6680,2022-08-01,0064100000TL1Fw,Closed Lost,New Business,317822.6165,2018-09-12,2022-07-07,Diversion Monitoring,Epic,Childrens Hospital,...,43205.0,2.0,694.0,2196.0,7282.0,85077078.0,3.652792e+09,2.048661e+09,0,1394
136,2019-11-01,0064100000a4MNO,Closed Lost,New Business,239333.4085,2018-10-08,2019-10-01,Diversion Monitoring,Epic,Health System,...,60201.0,6.0,1429.0,3596.0,6071.0,60357592.0,9.389305e+09,2.649645e+09,0,358
743,2020-05-01,0064100000a6Ou1,Closed Lost,New Business,348860.0021,2018-10-30,2020-04-24,Diversion Monitoring,Epic,Health System,...,52242.0,2.0,922.0,1854.0,8485.0,15469189.0,6.972739e+09,2.113914e+09,0,542


In [ ]:
missing_values = df_filtered.isnull().sum()
missing_values

Snapshot_Date                                 0
Opportunity_ID                                0
Opportunity_Stage                             0
Opportunity_Type                              0
Opportunity_Annual_Amount                    33
Opportunity_Created_Date                      0
Opportunity_Close_Date                        0
Opportunity_Product                           0
Account_Electronic_Medical_Record_System      0
Account_Hospital_Type                        73
Account_Fiscal_Year_End                     601
Account_State                                72
Account_Zip_Code                             73
Account_Number_of_Hospitals                 207
Account_Number_of_Beds                       74
Account_Number_of_Affiliated_Physicians      79
Account_Number_of_Employee                  353
Account_Cash_on_Hand                         97
Account_Total_Patient_Revenue                88
Account_Net_Patient_Revenue                  88
Target_Close_Q3_2023                    

In [ ]:
df_filtered = df_filtered.dropna(subset=['Opportunity_Annual_Amount'])

In [ ]:
df_filtered

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Account_Electronic_Medical_Record_System,Account_Hospital_Type,...,Account_Zip_Code,Account_Number_of_Hospitals,Account_Number_of_Beds,Account_Number_of_Affiliated_Physicians,Account_Number_of_Employee,Account_Cash_on_Hand,Account_Total_Patient_Revenue,Account_Net_Patient_Revenue,Target_Close_Q3_2023,Opportunity_Duration
3059,2021-04-01,0061K00000b3JMy,Closed Lost,New Business,167838.3211,2019-03-05,2021-03-30,Privacy Monitoring,Other,Short Term Acute Care Hospital,...,15243.0,NaN,296.0,684.0,NaN,21723248.0,1.471406e+09,3.705945e+08,0,756
8063,2023-01-01,0061K00000b3Qsm,Closed Won,New Business,383667.8517,2019-03-07,2022-12-02,Privacy Monitoring,Epic,Health System,...,38120.0,21.0,2708.0,3670.0,12464.0,160091963.0,1.345325e+10,2.802628e+09,0,1366
386,2019-12-01,0061K00000b3uDh,Closed Lost,New Business,177324.3041,2019-03-13,2019-11-11,Privacy Monitoring,Epic,Short Term Acute Care Hospital,...,81003.0,NaN,268.0,596.0,2382.0,7557239.0,2.771359e+09,4.952779e+08,0,243
405,2020-01-01,0061K00000bI3DY,Closed Won,New Business,224617.8303,2018-11-20,2019-12-12,Privacy Monitoring,Epic,Childrens Hospital,...,60611.0,NaN,364.0,1675.0,NaN,123858245.0,3.091602e+09,1.039410e+09,0,387
383,2019-12-01,0061K00000bJS0n,Closed Lost,New Business,166841.1412,2018-12-14,2019-11-04,Diversion Monitoring,Epic,Health System,...,75235.0,2.0,449.0,1208.0,3592.0,181059081.0,4.296671e+09,1.895079e+09,0,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2021-02-01,0064100000TKw69,Closed Lost,New Business,412829.9260,2018-09-11,2021-01-26,Diversion Monitoring,Epic,Health System,...,49503.0,10.0,1901.0,4000.0,NaN,982985681.0,7.869236e+09,3.290488e+09,0,868
6680,2022-08-01,0064100000TL1Fw,Closed Lost,New Business,317822.6165,2018-09-12,2022-07-07,Diversion Monitoring,Epic,Childrens Hospital,...,43205.0,2.0,694.0,2196.0,7282.0,85077078.0,3.652792e+09,2.048661e+09,0,1394
136,2019-11-01,0064100000a4MNO,Closed Lost,New Business,239333.4085,2018-10-08,2019-10-01,Diversion Monitoring,Epic,Health System,...,60201.0,6.0,1429.0,3596.0,6071.0,60357592.0,9.389305e+09,2.649645e+09,0,358
743,2020-05-01,0064100000a6Ou1,Closed Lost,New Business,348860.0021,2018-10-30,2020-04-24,Diversion Monitoring,Epic,Health System,...,52242.0,2.0,922.0,1854.0,8485.0,15469189.0,6.972739e+09,2.113914e+09,0,542


In [ ]:
X = df_filtered[["Opportunity_Stage", "Opportunity_Type", "Opportunity_Product", "Opportunity_Duration"]]
y = df_filtered['Target_Close_Q3_2023']

In [ ]:
amount = df_filtered["Opportunity_Annual_Amount"]

In [ ]:
one_hot_X = pd.get_dummies(X)
one_hot_X

,Opportunity_Duration,Opportunity_Stage_Closed Lost,Opportunity_Stage_Closed Won,Opportunity_Stage_Stage 0 - Prospecting,Opportunity_Stage_Stage 1 - Evaluating,Opportunity_Stage_Stage 2 - Scoping,Opportunity_Stage_Stage 3 - Pricing,Opportunity_Stage_Stage 4 - Verbal / VOC,Opportunity_Stage_Stage 5 - Contracting,Opportunity_Type_Existing Business,Opportunity_Type_New Business,Opportunity_Product_Compliance Analytics,Opportunity_Product_Diversion Monitoring,Opportunity_Product_Privacy Monitoring
3059,756,1,0,0,0,0,0,0,0,0,1,0,0,1
8063,1366,0,1,0,0,0,0,0,0,0,1,0,0,1
386,243,1,0,0,0,0,0,0,0,0,1,0,0,1
405,387,0,1,0,0,0,0,0,0,0,1,0,0,1
383,325,1,0,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,868,1,0,0,0,0,0,0,0,0,1,0,1,0
6680,1394,1,0,0,0,0,0,0,0,0,1,0,1,0
136,358,1,0,0,0,0,0,0,0,0,1,0,1,0
743,542,1,0,0,0,0,0,0,0,0,1,0,1,0


In [ ]:
predicted_probabilities = rf_model.predict_proba(one_hot_X)

In [ ]:
probs = predicted_probabilities[:,1]

In [ ]:
len(probs)

828

In [ ]:
one_hot_X["probs"] = probs

In [ ]:
df_new = one_hot_X

In [ ]:
df_new

,Opportunity_Duration,Opportunity_Stage_Closed Lost,Opportunity_Stage_Closed Won,Opportunity_Stage_Stage 0 - Prospecting,Opportunity_Stage_Stage 1 - Evaluating,Opportunity_Stage_Stage 2 - Scoping,Opportunity_Stage_Stage 3 - Pricing,Opportunity_Stage_Stage 4 - Verbal / VOC,Opportunity_Stage_Stage 5 - Contracting,Opportunity_Type_Existing Business,Opportunity_Type_New Business,Opportunity_Product_Compliance Analytics,Opportunity_Product_Diversion Monitoring,Opportunity_Product_Privacy Monitoring,probs
3059,756,1,0,0,0,0,0,0,0,0,1,0,0,1,0.0
8063,1366,0,1,0,0,0,0,0,0,0,1,0,0,1,0.0
386,243,1,0,0,0,0,0,0,0,0,1,0,0,1,0.0
405,387,0,1,0,0,0,0,0,0,0,1,0,0,1,0.0
383,325,1,0,0,0,0,0,0,0,0,1,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,868,1,0,0,0,0,0,0,0,0,1,0,1,0,0.0
6680,1394,1,0,0,0,0,0,0,0,0,1,0,1,0,0.0
136,358,1,0,0,0,0,0,0,0,0,1,0,1,0,0.0
743,542,1,0,0,0,0,0,0,0,0,1,0,1,0,0.0


In [ ]:
sales = 0
# df_thresh = df_new[df_new['probs']>=0.5]
for i in range(len(df_new["probs"])):
  if list(df_new["Opportunity_Product_Diversion Monitoring"])[i] == 1:
    sales += list(df_new["probs"])[i]*(list(amount)[i] - 8361.49)
  if list(df_new["Opportunity_Product_Privacy Monitoring"])[i] == 1:
    sales += list(df_new["probs"])[i]*(list(amount)[i] - 8040.58)

In [ ]:
print(sales)

1939884.7336387667


In [ ]:
df_pred = pd.read_csv("/content/drive/MyDrive/AI_hack/data/question_2_prob.csv")

In [ ]:
df_pred

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product
0,1/7/2023,0061K00000mR3oA,Stage 1 - Evaluating,New Business,107458.94,30/6/2023,26/6/2024,Privacy Monitoring
1,1/7/2023,0061K00000lUGm2,Stage 0 - Prospecting,Existing Business,37890.00,28/6/2023,15/12/2023,Diversion Monitoring
2,1/7/2023,0061K00000lUCBW,Stage 2 - Scoping,New Business,56763.26,26/6/2023,30/6/2024,Diversion Monitoring
3,1/7/2023,0061K00000lUBat,Stage 0 - Prospecting,New Business,61201.00,26/6/2023,29/12/2023,Privacy Monitoring
4,1/7/2023,0061K00000lU5Vh,Stage 0 - Prospecting,Existing Business,44500.00,21/6/2023,1/2/2024,Diversion Monitoring
...,...,...,...,...,...,...,...,...
147,1/7/2023,0061K00000h6jwx,Stage 3 - Pricing,New Business,75578.99,3/3/2021,18/1/2024,Diversion Monitoring
148,1/7/2023,0061K00000fvP5f,Stage 3 - Pricing,Existing Business,81073.32,16/9/2020,1/11/2023,Diversion Monitoring
149,1/7/2023,0061K00000ftCbz,Stage 3 - Pricing,New Business,33890.00,22/7/2020,23/12/2023,Diversion Monitoring
150,1/7/2023,0061K00000fNEfL,Stage 1 - Evaluating,Existing Business,30240.00,22/5/2020,23/5/2024,Privacy Monitoring


In [ ]:
df_pred['Snapshot_Date'] = pd.to_datetime(df_pred['Snapshot_Date'])
df_pred['Opportunity_Created_Date'] = pd.to_datetime(df_pred['Opportunity_Created_Date'])
df_pred['Opportunity_Close_Date'] = pd.to_datetime(df_pred['Opportunity_Close_Date'])

<ipython-input-497-41cd1f1376cc>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_pred['Opportunity_Created_Date'] = pd.to_datetime(df_pred['Opportunity_Created_Date'])
<ipython-input-497-41cd1f1376cc>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_pred['Opportunity_Close_Date'] = pd.to_datetime(df_pred['Opportunity_Close_Date'])


In [ ]:
Q3_2023_start = datetime(2023, 7, 1)
Q3_2023_end = datetime(2023, 9, 30)

In [ ]:
df_pred['Target_Close_Q3_2023'] = df_pred['Opportunity_Close_Date'].apply(lambda x: 1 if Q3_2023_start <= x <= Q3_2023_end else 0)
df_pred['Opportunity_Duration'] = (df_pred['Opportunity_Close_Date'] - df_pred['Opportunity_Created_Date']).dt.days

In [ ]:
df_pred = df_pred.dropna(subset=['Opportunity_Annual_Amount'])
df_pred

,Snapshot_Date,Opportunity_ID,Opportunity_Stage,Opportunity_Type,Opportunity_Annual_Amount,Opportunity_Created_Date,Opportunity_Close_Date,Opportunity_Product,Target_Close_Q3_2023,Opportunity_Duration
0,2023-01-07,0061K00000mR3oA,Stage 1 - Evaluating,New Business,107458.94,2023-06-30,2024-06-26,Privacy Monitoring,0,362
1,2023-01-07,0061K00000lUGm2,Stage 0 - Prospecting,Existing Business,37890.00,2023-06-28,2023-12-15,Diversion Monitoring,0,170
2,2023-01-07,0061K00000lUCBW,Stage 2 - Scoping,New Business,56763.26,2023-06-26,2024-06-30,Diversion Monitoring,0,370
3,2023-01-07,0061K00000lUBat,Stage 0 - Prospecting,New Business,61201.00,2023-06-26,2023-12-29,Privacy Monitoring,0,186
4,2023-01-07,0061K00000lU5Vh,Stage 0 - Prospecting,Existing Business,44500.00,2023-06-21,2024-01-02,Diversion Monitoring,0,195
...,...,...,...,...,...,...,...,...,...,...
147,2023-01-07,0061K00000h6jwx,Stage 3 - Pricing,New Business,75578.99,2021-03-03,2024-01-18,Diversion Monitoring,0,1051
148,2023-01-07,0061K00000fvP5f,Stage 3 - Pricing,Existing Business,81073.32,2020-09-16,2023-01-11,Diversion Monitoring,0,847
149,2023-01-07,0061K00000ftCbz,Stage 3 - Pricing,New Business,33890.00,2020-07-22,2023-12-23,Diversion Monitoring,0,1249
150,2023-01-07,0061K00000fNEfL,Stage 1 - Evaluating,Existing Business,30240.00,2020-05-22,2024-05-23,Privacy Monitoring,0,1462


In [ ]:
X = df_pred[["Opportunity_Stage", "Opportunity_Type", "Opportunity_Product", "Opportunity_Duration"]]
y = df_pred['Target_Close_Q3_2023']

In [ ]:
one_hot_X = pd.get_dummies(X)
one_hot_X

,Opportunity_Duration,Opportunity_Stage_Stage 0 - Prospecting,Opportunity_Stage_Stage 1 - Evaluating,Opportunity_Stage_Stage 2 - Scoping,Opportunity_Stage_Stage 3 - Pricing,Opportunity_Stage_Stage 4 - Verbal / VOC,Opportunity_Stage_Stage 5 - Contracting,Opportunity_Type_Existing Business,Opportunity_Type_New Business,Opportunity_Product_Compliance Analytics,Opportunity_Product_Diversion Monitoring,Opportunity_Product_Privacy Monitoring
0,362,0,1,0,0,0,0,0,1,0,0,1
1,170,1,0,0,0,0,0,1,0,0,1,0
2,370,0,0,1,0,0,0,0,1,0,1,0
3,186,1,0,0,0,0,0,0,1,0,0,1
4,195,1,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
147,1051,0,0,0,1,0,0,0,1,0,1,0
148,847,0,0,0,1,0,0,1,0,0,1,0
149,1249,0,0,0,1,0,0,0,1,0,1,0
150,1462,0,1,0,0,0,0,1,0,0,0,1


In [ ]:
one_hot_X.insert(loc = 1,
          column = 'Opportunity_Stage_Closed Lost',
          value = 0)
one_hot_X.insert(loc = 2,
          column = 'Opportunity_Stage_Closed Won',
          value = 0)

In [ ]:
one_hot_X

,Opportunity_Duration,Opportunity_Stage_Closed Lost,Opportunity_Stage_Closed Won,Opportunity_Stage_Stage 0 - Prospecting,Opportunity_Stage_Stage 1 - Evaluating,Opportunity_Stage_Stage 2 - Scoping,Opportunity_Stage_Stage 3 - Pricing,Opportunity_Stage_Stage 4 - Verbal / VOC,Opportunity_Stage_Stage 5 - Contracting,Opportunity_Type_Existing Business,Opportunity_Type_New Business,Opportunity_Product_Compliance Analytics,Opportunity_Product_Diversion Monitoring,Opportunity_Product_Privacy Monitoring
0,362,0,0,0,1,0,0,0,0,0,1,0,0,1
1,170,0,0,1,0,0,0,0,0,1,0,0,1,0
2,370,0,0,0,0,1,0,0,0,0,1,0,1,0
3,186,0,0,1,0,0,0,0,0,0,1,0,0,1
4,195,0,0,1,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,1051,0,0,0,0,0,1,0,0,0,1,0,1,0
148,847,0,0,0,0,0,1,0,0,1,0,0,1,0
149,1249,0,0,0,0,0,1,0,0,0,1,0,1,0
150,1462,0,0,0,1,0,0,0,0,1,0,0,0,1


In [ ]:
predicted_probabilities = rf_model.predict_proba(one_hot_X)

In [ ]:
probs = predicted_probabilities[:,1]

In [ ]:
for prob in probs:
  print(prob)

0.0
0.01
0.0
0.21
0.01
0.0
0.0
0.07019047619047619
0.02
0.01
0.69
0.57
0.03666666666666667
0.01
0.0
0.62
0.6591666666666666
0.0
0.0
0.59
0.0
0.0
0.0
0.0
0.01
0.0
0.0
0.0
0.0
0.01
0.0
0.0
0.0
0.0
0.0
0.65
0.0
0.0
0.0
0.0
0.6839166666666665
0.0
0.0
0.0
0.39
0.0
0.01
0.03
0.0
0.0
0.02
0.0
0.0
0.0
0.01
0.0
0.67
0.8564466366966361
0.99
0.0
0.0
0.0
0.03
0.67
0.07
0.25
0.63
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.01
0.04
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.62
0.9816928314489582
0.0
0.0
0.0
0.0
0.0
0.0
0.01
0.02
0.33
0.0
0.01
0.0
0.01
0.0
0.0
0.0
0.09
0.03
0.0
0.02
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.04
0.01
0.0
0.64
0.0
0.03
0.06
0.0
0.0
0.02
0.0
0.11
0.0
0.0
0.0
0.0
0.97
0.0
0.01
0.61
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.02
0.27
